# Convert foreign payments to USD with historical FX rates

For each type of payment that needs to be converted to USD, and instance of ```FatcatCalculator``` is defined, which is instantiated with the currency of the payments, and a list of dicts containing the payment dates and amounts.

In [1]:
import pandas as pd
from datetime import datetime

from expat_fatcat import QuandlRateConverterToUSD, FatcatCalculator, f2555

## Using QUANDL to get fx rates
If you have a quandl api key, you can get real exchange rates using the ```QuandlRateConverterToUSD```. For the below examples, the fake payments are now in EUR.

In [2]:
converter = QuandlRateConverterToUSD()
converter.get_rate('EUR', datetime(2019, 4, 18))

1.125

### Imputing missing rates

In [3]:
# Bank holiday: Good Friday 2019
converter.get_rate('EUR', datetime(2019, 4, 19))

/Users/pauldev/delo/projects/expat-fatcat/src/expat_fatcat/expat_fatcat.py:160: UserWarning: Date 2019-04-19 00:00:00 is invalid, trying before and after
  warnings.warn("Date {} is invalid, trying before and after".format(date))
/Users/pauldev/delo/projects/expat-fatcat/src/expat_fatcat/expat_fatcat.py:160: UserWarning: Date 2019-04-20 00:00:00 is invalid, trying before and after
  warnings.warn("Date {} is invalid, trying before and after".format(date))


1.125

In [4]:
# Good Friday rate is taken from the day before
converter.get_rate('EUR', datetime(2019, 4, 19)) == converter.get_rate('EUR', datetime(2019, 4, 18))

/Users/pauldev/delo/projects/expat-fatcat/src/expat_fatcat/expat_fatcat.py:160: UserWarning: Date 2019-04-19 00:00:00 is invalid, trying before and after
  warnings.warn("Date {} is invalid, trying before and after".format(date))
/Users/pauldev/delo/projects/expat-fatcat/src/expat_fatcat/expat_fatcat.py:160: UserWarning: Date 2019-04-20 00:00:00 is invalid, trying before and after
  warnings.warn("Date {} is invalid, trying before and after".format(date))


True

In [5]:
# Easter Sunday rate is taken from following Tuesday
converter.get_rate('EUR', datetime(2019, 4, 21)) == converter.get_rate('EUR', datetime(2019, 4, 23))

/Users/pauldev/delo/projects/expat-fatcat/src/expat_fatcat/expat_fatcat.py:160: UserWarning: Date 2019-04-21 00:00:00 is invalid, trying before and after
  warnings.warn("Date {} is invalid, trying before and after".format(date))
/Users/pauldev/delo/projects/expat-fatcat/src/expat_fatcat/expat_fatcat.py:160: UserWarning: Date 2019-04-22 00:00:00 is invalid, trying before and after
  warnings.warn("Date {} is invalid, trying before and after".format(date))
/Users/pauldev/delo/projects/expat-fatcat/src/expat_fatcat/expat_fatcat.py:160: UserWarning: Date 2019-04-20 00:00:00 is invalid, trying before and after
  warnings.warn("Date {} is invalid, trying before and after".format(date))
/Users/pauldev/delo/projects/expat-fatcat/src/expat_fatcat/expat_fatcat.py:160: UserWarning: Date 2019-04-19 00:00:00 is invalid, trying before and after
  warnings.warn("Date {} is invalid, trying before and after".format(date))


True

# Example payment data

This data is represents fake payment data in the currency "FOO".

In [6]:
payments = dict(
    date = [datetime(2018, 1, 15), datetime(2018, 2, 15), datetime(2018, 3, 15)],
    amount = [100, 100, 120]
)
payments = [dict(date='2018-01-15', amount=100), dict(date='2018-02-15', amount=150)]

In [7]:
payment_calculator = FatcatCalculator(converter)
print('Example payment EUR to USD calculation: {}'
      .format(payment_calculator(payments, 'EUR')))

Example payment EUR to USD calculation: 310.165


## Compute total foreign income  and housing deduction examples of f2555

In [8]:
salary = [
    dict(date='2017-01-20', amount=1000), 
    dict(date='2017-02-20', amount=1000), 
    dict(date='2017-03-20', amount=1500)
]

calculator = FatcatCalculator(converter)
f2555_line_25 = calculator(salary, 'EUR')
print('Example salary calculation, line 25 of f2555: {}'.format(f2555_line_25))

Example salary calculation, line 25 of f2555: 3737.6000000000004


In [9]:
housing = [
    dict(date='2017-01-20', amount=650),
    dict(date='2017-02-20', amount=650),
    dict(date='2017-03-20', amount=650)
]
f2555_line_34 = calculator(housing, 'EUR')
print('Example housing deduction calculation, line 34 of f2555: {}'.format(f2555_line_34))

Example housing deduction calculation, line 34 of f2555: 2080.0


## Putting things together with the function ```f2555```

In [10]:
data = [
    {'tag': 'salary', 'currency': 'EUR', 'payments': salary},
    {'tag': 'rent', 'currency': 'EUR', 'payments': housing}
]
f2555(converter, data)

{'salary': {'currency': 'EUR', 'amount': 3737.6000000000004},
 'rent': {'currency': 'EUR', 'amount': 2080.0}}